# CAPITULO 3

# PYTHON

Hay dos manera para definir un esquema.        
La primera es definir un esquema porgramaticamente para Dataframe con tres columnas

In [1]:
from pyspark.sql.types import *

In [2]:
schema = StructType([StructField("author", StringType(), False),
                    StructField("title", StringType(), False),
                    StructField("pages", IntegerType(), False)])

La segunda es usando DDL (Data definition Language)

In [6]:
schema1 = "author STRING, title STRING, pages INT"

Ejemplo 2 -- Rellenando la tabla

In [1]:
from pyspark.sql import SparkSession

In [7]:
esquema = "ID INT, First STRING, Last STRING, URL STRING, Published STRING, Hits INT, Campaigns ARRAY<STRING>"

In [3]:
data=[[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
      [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
      [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web", "twitter", "FB", "LinkedIn"]],
      [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,["twitter", "FB"]],
      [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
      [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]]

In [5]:
spark = (SparkSession
        .builder
        .appName("Ejemplo.2")
        .getOrCreate())

In [8]:
blogs_df=spark.createDataFrame(data, esquema)

In [13]:
blogs_df.show(truncate=False)
print("La tabla tiene %d filas" % (blogs_df.count()))

+---+---------+-------+-----------------+---------+-----+----------------------------+
|ID |First    |Last   |URL              |Published|Hits |Campaigns                   |
+---+---------+-------+-----------------+---------+-----+----------------------------+
|1  |Jules    |Damji  |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke   |Wenig  |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny    |Lee    |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata|Das    |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei    |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold  |Xin    |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+---------+-------+-----------------+---------+-----+----------------------------+

La tabla tiene 6 filas


In [15]:
print(blogs_df.printSchema())

root
 |-- ID: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)

None


In [16]:
blogs_df.schema

StructType(List(StructField(ID,IntegerType,true),StructField(First,StringType,true),StructField(Last,StringType,true),StructField(URL,StringType,true),StructField(Published,StringType,true),StructField(Hits,IntegerType,true),StructField(Campaigns,ArrayType(StringType,true),true)))

Ejercicios proyecciones y filtros

In [1]:
from pyspark.sql import SparkSession

In [2]:
import sys

In [3]:
from pyspark.sql import functions as F

In [4]:
from pyspark.sql.functions import * 

In [5]:
spark = (SparkSession
        .builder
        .appName("Ejemplo.3")
        .getOrCreate())

In [7]:
fire_file = "C:/LearningSparkV2-master/databricks-datasets/learning-spark-v2/sf-fire-calls.csv"

In [8]:
fire_df = (spark.read.format("csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .load(fire_file))

In [25]:
few_fire_df = (fire_df.select("IncidentNumber", "AvailableDtTm", "CallType").where(fire_df.CallType != "Medical Incident"))

In [27]:
few_fire_df.show(10, truncate = False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
|2003304       |01/11/2002 09:58:53 AM|Alarms        |
|2003382       |01/11/2002 02:59:04 PM|Structure Fire|
|2003408       |01/11/2002 04:09:08 PM|Structure Fire|
|2003408       |01/11/2002 04:09:08 PM|Structure Fire|
|2003408       |01/11/2002 04:09:08 PM|Structure Fire|
+--------------+----------------------+--------------+
only showing top 10 rows



In [10]:
fire_df.select("CallType").show(2)

+----------------+
|        CallType|
+----------------+
|  Structure Fire|
|Medical Incident|
+----------------+
only showing top 2 rows



In [11]:
(fire_df.select("CallType").where(fire_df.CallType.isNotNull()).agg(countDistinct("CallType").alias("DistinctCallTypes")).show())

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [13]:
(fire_df.select("CallType").where(fire_df.CallType.isNotNull()).distinct().show(10, False))

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



In [14]:
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")

In [16]:
(new_fire_df.select("ResponseDelayedinMins").where(new_fire_df.ResponseDelayedinMins>5).show(5, False))

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



In [17]:
fire_ts_df = (new_fire_df.withColumn("IncidentDate", to_timestamp(new_fire_df.CallDate, "MM/dd/yyyy")).drop("CallDate").withColumn("OnWatchDate", to_timestamp(new_fire_df.WatchDate, "MM/dd/yyyy")).drop("WatchDate").withColumn("AvailableDtTs", to_timestamp(new_fire_df.AvailableDtTm, "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm"))

In [19]:
(fire_ts_df.select("IncidentDate", "OnWatchDate", "AvailableDtTs").show(5, False))

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTs      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [20]:
(fire_ts_df.select("IncidentDate").distinct().orderBy(year("IncidentDate")).show())

+-------------------+
|       IncidentDate|
+-------------------+
|2000-12-13 00:00:00|
|2000-07-18 00:00:00|
|2000-12-15 00:00:00|
|2000-10-09 00:00:00|
|2000-05-17 00:00:00|
|2000-05-06 00:00:00|
|2000-11-12 00:00:00|
|2000-10-07 00:00:00|
|2000-10-23 00:00:00|
|2000-08-12 00:00:00|
|2000-04-22 00:00:00|
|2000-08-20 00:00:00|
|2000-12-03 00:00:00|
|2000-08-01 00:00:00|
|2000-06-10 00:00:00|
|2000-07-21 00:00:00|
|2000-09-04 00:00:00|
|2000-08-30 00:00:00|
|2000-09-28 00:00:00|
|2000-06-27 00:00:00|
+-------------------+
only showing top 20 rows



In [22]:
(fire_ts_df.select("CallType").where(fire_ts_df.CallType.isNotNull()).groupBy("CallType").count().orderBy("count", ascending = False).show(10, False))

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [23]:
(fire_ts_df.select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"), F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins")).show())

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|        3.8923641541750134|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



Generic Rows

In [24]:
from pyspark.sql import Row

In [29]:
row = Row(350, True, "Learning Spark 2E", None)

In [36]:
row[0]

350

In [37]:
row[1]

True

In [38]:
row[2]

'Learning Spark 2E'

# SCALA 

Hay dos manera iual que en Python.            
La primera es programaticamente

In [1]:
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002861.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1622632040127)
SparkSession available as 'spark'


import org.apache.spark.sql.types._


In [2]:
val schema = StructType(Array(StructField("author", StringType, false),
                             StructField("title", StringType, false),
                             StructField("pages", IntegerType, false)))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(author,StringType,false), StructField(title,StringType,false), StructField(pages,IntegerType,false))


La segunda es usando DDL (Data Definition Language)

In [3]:
val schema1 = "author STRING, title STRING, pages INT"

schema1: String = author STRING, title STRING, pages INT


In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002861.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1622644520753)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._


In [3]:
val esquema = StructType(Array(StructField("Id", IntegerType, false),
 StructField("First", StringType, false),
 StructField("Last", StringType, false),
 StructField("Url", StringType, false),
 StructField("Published", StringType, false),
 StructField("Hits", IntegerType, false),
 StructField("Campaigns", ArrayType(StringType), false)))

esquema: org.apache.spark.sql.types.StructType = StructType(StructField(Id,IntegerType,false), StructField(First,StringType,false), StructField(Last,StringType,false), StructField(Url,StringType,false), StructField(Published,StringType,false), StructField(Hits,IntegerType,false), StructField(Campaigns,ArrayType(StringType,true),false))


In [4]:
val dt = "C:/LearningSparkV2-master/databricks-datasets/learning-spark-v2/blogs.json"

dt: String = C:/LearningSparkV2-master/databricks-datasets/learning-spark-v2/blogs.json


In [5]:
val blogsDF = spark.read.schema(esquema).json(dt)

blogsDF: org.apache.spark.sql.DataFrame = [Id: int, First: string ... 5 more fields]


In [6]:
blogsDF.show(false)

+---+---------+-------+-----------------+---------+-----+----------------------------+
|Id |First    |Last   |Url              |Published|Hits |Campaigns                   |
+---+---------+-------+-----------------+---------+-----+----------------------------+
|1  |Jules    |Damji  |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke   |Wenig  |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny    |Lee    |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata|Das    |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei    |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold  |Xin    |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+---------+-------+-----------------+---------+-----+----------------------------+



In [7]:
println(blogsDF.printSchema())

root
 |-- Id: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)

()


In [8]:
println(blogsDF.schema)

StructType(StructField(Id,IntegerType,true), StructField(First,StringType,true), StructField(Last,StringType,true), StructField(Url,StringType,true), StructField(Published,StringType,true), StructField(Hits,IntegerType,true), StructField(Campaigns,ArrayType(StringType,true),true))


Ejemplos actuando sobre las columnas

In [9]:
blogsDF.columns

res3: Array[String] = Array(Id, First, Last, Url, Published, Hits, Campaigns)


In [11]:
blogsDF.col("id")

res5: org.apache.spark.sql.Column = id


In [22]:
blogsDF.select((expr("Hits * 2")), col("Hits")).show(2)

+----------+----+
|(Hits * 2)|Hits|
+----------+----+
|      9070|4535|
|     17816|8908|
+----------+----+
only showing top 2 rows



In [23]:
blogsDF.select((col("Hits") * 2), col("Hits")).show(2)

+----------+----+
|(Hits * 2)|Hits|
+----------+----+
|      9070|4535|
|     17816|8908|
+----------+----+
only showing top 2 rows



In [18]:
blogsDF.withColumn("Big Hitters", (expr("Hits > 10000"))).show()

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|      false|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|      false|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|      false|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|       true|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|       true|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|       true|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+



In [19]:
blogsDF.withColumn("AuthorsID", (concat(expr("First"), expr("Last"), expr("Id")))).select(col("AuthorsID")).show(4)

+-------------+
|    AuthorsID|
+-------------+
|  JulesDamji1|
| BrookeWenig2|
|    DennyLee3|
|TathagataDas4|
+-------------+
only showing top 4 rows



Las siguientes funciones devuelven los mismos resultados

In [26]:
blogsDF.select(expr("Hits")).show(2)
blogsDF.select(col("Hits")).show(2)
blogsDF.select("Hits").show(2)

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows



Las dos expresiones sort (sirve para ordenar, en este caso descendentemente), hacen lo mismo solo que en la segunda el caracter $ que convierte el valor Id en una columna


In [29]:
blogsDF.sort(col("Id")desc).show()
blogsDF.sort($"Id".desc).show

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+

Ejemplos actuando sobre filas 

In [30]:
import org.apache.spark.sql.Row

import org.apache.spark.sql.Row


In [31]:
val blogRow = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", Array("twitter", "LinkedIn"))

blogRow: org.apache.spark.sql.Row = [6,Reynold,Xin,https://tinyurl.6,255568,3/2/2015,[Ljava.lang.String;@1b54aea0]


In [33]:
blogRow(1)

res24: Any = Reynold


In [34]:
val rows = Seq(("Matei Zahara", "CA"), ("Reynold Xin", "CA"))

rows: Seq[(String, String)] = List((Matei Zahara,CA), (Reynold Xin,CA))


In [35]:
val authorsDF = rows.toDF("Author", "State")

authorsDF: org.apache.spark.sql.DataFrame = [Author: string, State: string]


In [36]:
authorsDF.show()

+------------+-----+
|      Author|State|
+------------+-----+
|Matei Zahara|   CA|
| Reynold Xin|   CA|
+------------+-----+



Creando un DataSet: para poder crear el DataSer, hay que saber de antemano los tipos de datos que va a contener, el esquema

In [1]:
case class DeviceIoTData (battery_level: Long, c02_level: Long,
cca2: String, cca3: String, cn: String, device_id: Long,
device_name: String, humidity: Long, ip: String, latitude: Double,
lcd: String, longitude: Double, scale:String, temp: Long,
timestamp: Long)

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002861.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1623054334674)
SparkSession available as 'spark'


defined class DeviceIoTData


In [4]:
val ds = spark.read.json("C:/LearningSparkV2-master/databricks-datasets/learning-spark-v2/iot_devices.json").as[DeviceIoTData]

ds: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]


In [5]:
ds.show(5, false)

+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name          |humidity|ip           |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|8            |868      |US  |USA |United States|1        |meter-gauge-1xbYRYcj |51      |68.161.225.1 |38.0    |green |-97.0    |Celsius|34  |1458444054093|
|7            |1473     |NO  |NOR |Norway       |2        |sensor-pad-2n2Pea    |70      |213.161.254.1|62.47   |red   |6.15     |Celsius|11  |1458444054119|
|2            |1556     |IT  |ITA |Italy        |3        |device-mac-36TWSKiT  |44      |88.36.5.1    |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|6            |1080     |US  |USA |United States|4  

Operacion con DataSets

In [8]:
val filterTempDS = ds.filter({d => {d.temp > 30 && d.humidity > 70}})

filterTempDS: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]


filterTempDS.show(5, false)

Otro DataSet mas pequeño

In [11]:
case class DeviceTempByCountry(temp: Long, device_name: String, device_id: Long,
 cca3: String)

defined class DeviceTempByCountry


val dsTemp = ds .filter(d => {d.temp > 25}) .map(d => (d.temp, d.device_name, d.device_id, d.cca3)) .toDF("temp", "device_name", "device_id", "cca3") .as[DeviceTempByCountry] dsTemp.show(5, false)

Otra manera de crear el DataSet

In [12]:
val dsTemp2 = ds
 .select($"temp", $"device_name", $"device_id", $"device_id", $"cca3")
 .where("temp > 25")
 .as[DeviceTempByCountry]

dsTemp2: org.apache.spark.sql.Dataset[DeviceTempByCountry] = [temp: bigint, device_name: string ... 3 more fields]


In [13]:
dsTemp2.show(5, false)

+----+---------------------+---------+---------+----+
|temp|device_name          |device_id|device_id|cca3|
+----+---------------------+---------+---------+----+
|34  |meter-gauge-1xbYRYcj |1        |1        |USA |
|28  |sensor-pad-4mzWkz    |4        |4        |USA |
|27  |sensor-pad-6al7RTAobR|6        |6        |USA |
|27  |sensor-pad-8xUD6pzsQI|8        |8        |JPN |
|26  |sensor-pad-10BsywSYUF|10       |10       |USA |
+----+---------------------+---------+---------+----+
only showing top 5 rows



Para mirar solo la primera fila del DataSet

In [14]:
val device = dsTemp2.first()

device: DeviceTempByCountry = DeviceTempByCountry(34,meter-gauge-1xbYRYcj,1,USA)


In [15]:
println(device)

DeviceTempByCountry(34,meter-gauge-1xbYRYcj,1,USA)


In [16]:
dsTemp2.first()

res6: DeviceTempByCountry = DeviceTempByCountry(34,meter-gauge-1xbYRYcj,1,USA)


DataFrames vs Datasets

• If you want to tell Spark what to do, not how to do it, use DataFrames or Datasets.     
• If you want rich semantics, high-level abstractions, and DSL operators, use DataFrames or Datasets.     
• If you want strict compile-time type safety and don’t mind creating multiple case classes for a specific Dataset[T], use Datasets.     
• If your processing demands high-level expressions, filters, maps, aggregations, computing averages or sums, SQL queries, columnar access, or use of relational operators on semi-structured data, use DataFrames or Datasets.     
• If your processing dictates relational transformations similar to SQL-like queries, use DataFrames.     
• If you want to take advantage of and benefit from Tungsten’s efficient serializa‐ tion with Encoders, , use Datasets.     
• If you want unification, code optimization, and simplification of APIs across Spark components, use DataFrames.    
• If you are an R user, use DataFrames.     
• If you are a Python user, use DataFrames and drop down to RDDs if you need more control.     
• If you want space and speed efficiency, use DataFrames.